## Парсинг данных с сайта на Python

### Определения

Парсинг - это процесс сбора данных с последующей их обработкой и анализом.

Программа, которая занимается парсингом, называют - парсер.


### Условие задачи

С сайта ( https://habr.com/ru/search/ ) необходимо построить исходный набор данных (.csv или .xml). Набор данных должен включать __названия, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статей из Интернет-ресурсов__. Подготовленный набор данных должен содержать сведения о всех номинантах конкурса. Разработанный парсер должен извлекать гиперссылки из начальной страницы с последующим обходом всех страниц по полученным ссылкам и извлечением их содержимого. Можно дополнить набор какими-либо другими данными, если они могут быть полезны для дальнейшего исследования.


### Этапы парсинга

1. Поиск данных
2. Получение информации
3. Сохранение данных

### Подключение библиотек

In [1]:
from bs4 import BeautifulSoup as bs

Beautiful Soup - это библиотека Python для извлечения данных из HTML и XML файлов. 

In [11]:
import requests

Библиотека requests является стандартным инструментом для составления HTTP-запросов в Python.

In [12]:
import pandas as pd

### Получение информаций

In [13]:
# GET - запрос
url = 'https://habr.com/ru/all/' # страница со всеми статьями 
page = requests.get(url)

Метод __requests.get(url)__ из библиотеки requests в Python выполняет HTTP-запрос типа GET по указанному URL. Этот запрос используется для получения данных с веб-страницы или API, в нашем случае из страницы habr.

In [14]:
page.status_code

200

Если вызвать __page.status_code__, то получим статус состояния HTTP. например, 200 — успешно, 404 — страница не найдена, 500 — ошибка сервера 


In [15]:
soup = bs(page.text, 'html.parser')

__bs__ — это сокращение от BeautifulSoup, основного класса библиотеки Beautiful Soup.

__bs(page.text, 'html.parser')__ создаёт объект BeautifulSoup, который парсит HTML-код из page.text с использованием указанного парсера.

__'html.parser'__ — это встроенный парсер Python, который не требует установки дополнительных библиотек.а.

In [16]:
page.text

'<!DOCTYPE html>\n<html lang="ru">\n\n  <head>\n    <title>Все статьи подряд &#x2F; Хабр</title>\n<link rel="image_src" href="/img/habr_ru.png" data-hid="2a79c45">\n<link href="https://habr.com/ru/articles/" rel="canonical" data-hid="e3fa780">\n<link href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate" data-hid="7d51b8a">\n<link href="https://habr.com/en/articles/" hreflang="en" rel="alternate" data-hid="7d51b8a">\n<meta itemprop="image" content="/img/habr_ru.png">\n<meta property="og:image" content="/img/habr_ru.png">\n<meta property="og:image:width" content="1200">\n<meta property="og:image:height" content="630">\n<meta property="aiturec:image" content="/img/habr_ru.png">\n<meta name="twitter:image" content="/img/habr_ru.png">\n<meta property="vk:image" content="/img/habr_ru.png?format=vk">\n<meta property="fb:app_id" content="444736788986613">\n<meta property="fb:pages" content="472597926099084">\n<meta name="twitter:card" content="summary_large_image">\n<meta name="tw

In [17]:
soup

<!DOCTYPE html>

<html lang="ru">
<head>
<title>Все статьи подряд / Хабр</title>
<link data-hid="2a79c45" href="/img/habr_ru.png" rel="image_src"/>
<link data-hid="e3fa780" href="https://habr.com/ru/articles/" rel="canonical"/>
<link data-hid="7d51b8a" href="https://habr.com/ru/articles/" hreflang="ru" rel="alternate"/>
<link data-hid="7d51b8a" href="https://habr.com/en/articles/" hreflang="en" rel="alternate"/>
<meta content="/img/habr_ru.png" itemprop="image"/>
<meta content="/img/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="630" property="og:image:height"/>
<meta content="/img/habr_ru.png" property="aiturec:image"/>
<meta content="/img/habr_ru.png" name="twitter:image"/>
<meta content="/img/habr_ru.png?format=vk" property="vk:image"/>
<meta content="444736788986613" property="fb:app_id"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="summary_large_image" name="twitter:card"/>
<meta content="@habr_com" name=

Создадим словарь, в который будем записывать данные по заданию: название статьи, описание, рейтинг и сферу деятельности компаний, дату публикации, а также текст статьи из Интернет-ресурса

In [18]:
result_list = {'Заголовок': [], 'Название компании': [], 'Описание': [], 'Рейтинг': [], 'Категория': [], 'Дата': [], 'Текст': []}

### Алгоритм

Суть алгоритма заключается в переборе страниц, и переходе на "вложенные" страницы, то есть у нас есть основная страница https://habr.com/ru/all/, мы перебираем несколько стараниц с page1 до page10. На каждой странице есть статьи, записываем их в список, чтобы перейти по ним используем  _-i.a.get('href')-_  то есть берём значение из href этого заголовка. Далее находим классы элементов которые нам нужны, и записываем их в результат.

In [19]:
pagenum = 1
for i in range(3):
    url = 'https://habr.com/ru/articles/page' + str(pagenum) + '/' # переход на ссылуку с определённым номером сраницы
    page = requests.get(url)
    soup = bs(page.text, 'html.parser')
    titles = soup.find_all('h2', class_='tm-title tm-title_h2')# получаем заголовки всех статей на этой странице
    
    for i in titles:
        # переход на страницу статьи
        url = 'https://habr.com' + str(i.a.get('href')) 
        page = requests.get(url)
        soup = bs(page.text, 'html.parser')
        
        name_company = soup.find('a', class_='tm-company-snippet__title')# получаем название компаний
        desc_company = soup.find('div', class_='tm-company-snippet__description')# получаем описание компаний
        
        if (name_company is not None): #если на странице присутсвует компания
        
            result_list['Заголовок'].append(i.text) # записываем название статьи
            result_list['Название компании'].append(name_company.text) # записываем название компании
            result_list['Описание'].append(desc_company.text) # записываем описание компании
            
            datepub = soup.find('span', class_='tm-article-datetime-published') # находим дату публикаций
            result_list['Дата'].append(datepub.time['datetime'][0: 10]) # записываем дату публикаций
            
            # текст статьи
            try:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-2').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            except:
                textpub = soup.find('div', class_='article-formatted-body article-formatted-body article-formatted-body_version-1').get_text()
                textpub = textpub.replace('\n', ' ').replace('\t', ' ').replace('\xa0', ' ').replace('\u200e', ' ').replace('\r', ' ')
            result_list['Текст'].append(textpub)
            
            # переход на страницу компании
            url = 'https://habr.com' + str(name_company.get('href'))
            page = requests.get(url)
            soup = bs(page.text, 'html.parser')
            
            #записываем рейтинг
            rating = soup.find('span', class_='tm-votes-lever__score-counter tm-votes-lever__score-counter_rating tm-votes-lever__score-counter')

            if(rating is None):
                result_list['Рейтинг'].append('0')
            else:
                result_list['Рейтинг'].append((rating.text).strip())
               
             #записываем отрасли компаний
            fieldtext = ""
            fields = soup.find_all('a', 'tm-company-profile__categories-text')
            for field in fields:
                fieldtext = fieldtext + ((field.text).strip()) + ", "
            if (fields is None):
                result_list['Категория'].append(None)
            else:
                result_list['Категория'].append(fieldtext[0:-2])
            
    pagenum += 1

In [20]:
result_list

{'Заголовок': ['Готовимся к встрече с Python 3.14: разбор суперсил',
  'Разворачиваем GitLab за пару кликов',
  'Я — редактор. И боюсь, что меня заменит нейросеть',
  'Искусственный интеллект в киберзащите',
  'Termidesk Connect — следующий уровень управления инфраструктурой',
  'Чёрный ящик раскрыт: как инъекция промта заставляет ИИ говорить всё и вытягивает системный запрос',
  'История C#: эволюция версий, .NET, Unity, Blazor, MAUI',
  'Как мы делаем IAM для облака MWS. От пользователей и до сервисных агентов',
  'Через тернии к 5 тысячам звёзд на гитхабе – мой путь опенсорсера',
  'Jqwik: обзор тестирования на основе свойств в UI и API',
  'Как мы перевели аналитику внутренних сервисов Авито на собственное решение',
  'Будьте добры, помедленнее! Я записываю… или Как мы приручили нейросеть для видеозвонков',
  'Goodbye, ThinkPad…',
  'Как мы научились сохранять тембр и\xa0интонацию спикера при переводе видео в\xa0Яндекс\xa0Браузере',
  'MP3 устарел. Будущее за современными lossless-

In [21]:
print("Количество нулевых значений в: ")
for i in result_list:
    print( i + " - " + str(result_list[i].count(None)))

Количество нулевых значений в: 
Заголовок - 0
Название компании - 0
Описание - 0
Рейтинг - 0
Категория - 0
Дата - 0
Текст - 0


### Сохранение данных

In [22]:
file_name = 'habr.csv'
df = pd.DataFrame(data=result_list)
df.to_csv(file_name)

In [23]:
df.head()

,Заголовок,Название компании,Описание,Рейтинг,Категория,Дата,Текст
0,Готовимся к встрече с Python 3.14: разбор супе...,МТС,Про жизнь и развитие в IT,2354.2,"Связь и телекоммуникации, Мобильные технологии...",2025-04-28,"Всем привет! С вами Леша Жиряков, техлид backe..."
1,Разворачиваем GitLab за пару кликов,OTUS,Цифровые навыки от ведущих экспертов,529.34,"Консалтинг и поддержка, Рекрутинг и HR, Произв...",2025-04-28,GitLab является одним из самых популярных инст...
2,"Я — редактор. И боюсь, что меня заменит нейросеть",Minervasoft,Платформа управления знаниями для команд и GenAI,279.27,"Программное обеспечение, Поисковые технологии,...",2025-04-28,"Я помню, как весной 2024 года ИИ едва выдавал ..."
3,Искусственный интеллект в киберзащите,Positive Technologies,Лидер результативной кибербезопасности,414.34,"Веб-разработка, Программное обеспечение, Инфор...",2025-04-28,За последние несколько лет статус технологий и...
4,Termidesk Connect — следующий уровень управлен...,Группа Астра,Компания,106.17,"Программное обеспечение, Аппаратное обеспечени...",2025-04-28,Сейчас нелегко найти организацию — будь то цве...


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Заголовок          30 non-null     object
 1   Название компании  30 non-null     object
 2   Описание           30 non-null     object
 3   Рейтинг            30 non-null     object
 4   Категория          30 non-null     object
 5   Дата               30 non-null     object
 6   Текст              30 non-null     object
dtypes: object(7)
memory usage: 1.8+ KB
